In [3]:
from __future__ import print_function
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding
from keras.layers import Conv1D, GlobalMaxPooling1D
from keras.layers import LSTM
from keras.preprocessing.text import text_to_word_sequence
from keras.preprocessing.text import hashing_trick
from keras.preprocessing.text import one_hot

from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding
from keras.layers import Conv1D, GlobalMaxPooling1D, MaxPooling1D

import sys
import pandas as pd
import numpy as np
import re
from sklearn import linear_model
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn import svm

from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier



from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

from sklearn.naive_bayes import GaussianNB

import nltk
from nltk import word_tokenize          
from nltk.stem.porter import PorterStemmer

from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MultiLabelBinarizer

from keras import metrics
from keras import losses

/home/nikepupu/.local/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [4]:
t = pd.read_csv("./reports/data.csv")

In [8]:
temp = t['labels'].tolist()
res = [text_to_word_sequence(a) for a in temp]
res2 = [one_hot(a,10) for a in temp]
mlb = MultiLabelBinarizer()
y = mlb.fit_transform(res)
print(y)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 1 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [39]:
l = t['problem-statement'].tolist()
ps = []
regex = re.compile('^a-zA-Z0-9')
def clean_s(input_str):
    return regex.sub(' ', input_str)

for i in range(len(l)):
    try:
        ps.append(clean_s(l[i]))
    except:
        ps.append("")


["Theatre Square in the capital city of Berland has a rectangular shape with the size n\xe2\x80\x89\xc3\x97\xe2\x80\x89m meters. On the occasion of the city's anniversary, a decision was taken to pave the Square with square granite flagstones. Each flagstone is of the size a\xe2\x80\x89\xc3\x97\xe2\x80\x89a.What is the least number of flagstones needed to pave the Square? It's allowed to cover the surface larger than the Theatre Square, but the Square has to be covered. It's not allowed to break the flagstones. The sides of flagstones should be parallel to the sides of the Square.",
 'In the popular spreadsheets systems (for example, in Excel) the following numeration of columns is used. The first column has number A, the second \xe2\x80\x94 number B, etc. till column 26 that is marked by Z. Then there are two-letter numbers: column 27 has number AA, 28 \xe2\x80\x94 AB, column 52 is marked by AZ. After ZZ there follow three-letter numbers, etc.The rows are marked by integer numbers sta

In [38]:
max_features = 20000
maxlen = 400
batch_size = 32
embedding_dims = 500
filters = 250
kernel_size = 3
hidden_dims = 2500
epochs = 2


temp =  [hashing_trick(a, max_features) for a in ps]
X_train, X_test, y_train, y_test = train_test_split(temp,y, test_size=0.2,random_state=42)
x_train = sequence.pad_sequences(X_train)
x_test = sequence.pad_sequences(X_test)
print(temp[0])
print(hashing_trick("Hello world!", max_features))

[10436, 15596, 314, 14952, 19938, 11278, 18243, 15449, 7097, 18177, 16949, 5546, 1105, 14952, 7081, 15860, 2907, 18235, 14952, 382, 18243, 14952, 9909, 8264, 18177, 10171, 10700, 13379, 16223, 12742, 14952, 15596, 1105, 15596, 6509, 13318, 3360, 3033, 301, 18243, 14952, 7081, 8710, 9917, 301, 14952, 5466, 12773, 18243, 13318, 14390, 16223, 12742, 14952, 15596, 19977, 19958, 16223, 8974, 14952, 11342, 7739, 6208, 14952, 10436, 15596, 3581, 14952, 15596, 7097, 16223, 3080, 4351, 19977, 10869, 19958, 16223, 18902, 14952, 13318, 14952, 18756, 18243, 13318, 4706, 3080, 2052, 16223, 14952, 18756, 18243, 14952, 15596]
[13432, 9189]


In [9]:
model = RandomForestClassifier()

stemmer = PorterStemmer()
def stem_tokens(tokens, stemmer):
    stemmed = []
    for item in tokens:
        stemmed.append(stemmer.stem(item))
    return stemmed

def tokenize(text):
    tokens = nltk.word_tokenize(text)
    stems = stem_tokens(tokens, stemmer)
    return stems

vectorizer = TfidfVectorizer(tokenizer=tokenize, stop_words="english",analyzer = 'word')

features = vectorizer.fit_transform(ps)

X_train, X_test, y_train, y_test = train_test_split(features, y, test_size=0.2,random_state=42)
model.fit(X_train,  y_train)
y_pred = model.predict(X_test)


In [12]:
y_pred_new = [[round(i) for i in line] for line in y_pred]
confusion_matrix = np.zeros(shape=(40,40) )
for i ,j in zip(y_pred_new, y_test):
    cnt = -1
    for a,b in zip(i,j):
        cnt = cnt+1
        if a == b and a == 1:
            confusion_matrix[cnt, cnt] = confusion_matrix[cnt, cnt]+1
       
            

c  = np.zeros(shape=(40,1))
for i in y_test:
    cnt = -1
    for j in i:
        cnt = cnt+1
        if j == 1:
            c[cnt] = c[cnt]+1


print(np.diag(confusion_matrix))
print(c.flatten())
np.diag(confusion_matrix).reshape(40,1) / c.flatten()

[ 0. 13.  2.  7.  0. 13. 10. 24. 16.  0. 32.  4.  0.  0.  0.  3.  5.  0.
 12. 40.  2. 65.  0. 25.  0.  0.  0. 14.  4.  0.  0.  4.  8.  8. 10.  0.
  0.  0.  6.  2.]
[  4.  78.  28. 144.   4.  43.  85. 124.  73.  12. 168.  21.   9.   5.
  15.  19.  38.   2.  70. 154.  12. 242.   2. 160.  12.   2.   2.  44.
  21.   4.   1.  25.  72.  27.  43.   3.   5.   2.  46.  28.]


array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [3.25      , 0.16666667, 0.46428571, ..., 6.5       , 0.2826087 ,
        0.46428571],
       [0.5       , 0.02564103, 0.07142857, ..., 1.        , 0.04347826,
        0.07142857],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [1.5       , 0.07692308, 0.21428571, ..., 3.        , 0.13043478,
        0.21428571],
       [0.5       , 0.02564103, 0.07142857, ..., 1.        , 0.04347826,
        0.07142857]])

In [82]:
print('Build model...')
model = Sequential()

# we start off with an efficient embedding layer which maps
# our vocab indices into embedding_dims dimensions


model.add(Embedding(max_features, 128))
model.add(Dropout(0.2))

# we add a Convolution1D, which will learn filters
# word group filters of size filter_length:
model.add(Conv1D(filters,
                 kernel_size,
                 padding='valid',
                 activation='relu',
                 strides=1))
# we use max pooling:
model.add(GlobalMaxPooling1D())

# We add a vanilla hidden layer:
model.add(Dense(hidden_dims))
model.add(Dropout(0.2))
model.add(Activation('relu'))

# We project onto a single unit output layer, and squash it with a sigmoid:
model.add(Dense(40))
model.add(Activation('sigmoid'))

def top_3_accuracy(y_true, y_pred):
    return metrics.top_k_categorical_accuracy(y_true, y_pred, k=3)

def top_5_accuracy(y_true, y_pred):
    return metrics.top_k_categorical_accuracy(y_true, y_pred, k=5)
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=[top_3_accuracy])
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=50, 
validation_data=(x_test, y_test))

y_pred = model.predict(x_test)



Build model...


ValueError: The first layer in a Sequential model must get an `input_shape` or `batch_input_shape` argument.

In [15]:
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(40, activation='sigmoid'))

# try using different optimizers and different optimizer configs
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print('Train...')

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=10,
          validation_data=(x_test, y_test))

score, acc = model.evaluate(x_test, y_test,
                            batch_size=batch_size)
print('Test score:', score)
print('Accuracy:', acc)
y_pred = model.predict(x_test)

Train...
Train on 3421 samples, validate on 856 samples
Epoch 1/10
3421/3421 [==============================] - 131s 38ms/step - loss: 0.2470 - acc: 0.9341 - val_loss: 0.1755 - val_acc: 0.9460
Epoch 2/10
3421/3421 [==============================] - 131s 38ms/step - loss: 0.1785 - acc: 0.9451 - val_loss: 0.1744 - val_acc: 0.9460
Epoch 3/10
3421/3421 [==============================] - 132s 39ms/step - loss: 0.1773 - acc: 0.9451 - val_loss: 0.1732 - val_acc: 0.9460
Epoch 4/10
3421/3421 [==============================] - 131s 38ms/step - loss: 0.1747 - acc: 0.9452 - val_loss: 0.1721 - val_acc: 0.9462
Epoch 5/10
3421/3421 [==============================] - 132s 39ms/step - loss: 0.1706 - acc: 0.9454 - val_loss: 0.1714 - val_acc: 0.9462
Epoch 6/10
3421/3421 [==============================] - 130s 38ms/step - loss: 0.1646 - acc: 0.9464 - val_loss: 0.1712 - val_acc: 0.9457
Epoch 7/10
3421/3421 [==============================] - 131s 38ms/step - loss: 0.1574 - acc: 0.9478 - val_loss: 0.1716 - v

In [16]:
y_pred[y_pred>=0.5] = 1
y_pred[y_pred<0.5] = 0
print(y_pred[0])
 

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [77]:
y_pred_new = [[round(i) for i in line] for line in y_pred]
confusion_matrix = np.zeros(shape=(40,40) )
for i ,j in zip(y_pred_new, y_test):
    cnt = -1
    for a,b in zip(i,j):
        cnt = cnt+1
        if a == b and a == 1:
            confusion_matrix[cnt, cnt] = confusion_matrix[cnt, cnt]+1
       
            

c  = np.zeros(shape=(40,1))
for i in y_test:
    cnt = -1
    for j in i:
        cnt = cnt+1
        if j == 1:
            c[cnt] = c[cnt]+1


print(np.diag(confusion_matrix))
print(c)
np.diag(confusion_matrix).reshape(40,1) / c


[ 0. 24.  5. 19.  0. 22. 21. 49. 24.  0. 67.  8.  0.  0.  2.  9. 17.  0.
 21. 49.  3. 79.  0. 49.  0.  0.  0. 17.  7.  0.  0.  5. 14. 18. 19.  0.
  0.  0. 23.  9.]
[[  4.]
 [ 78.]
 [ 28.]
 [144.]
 [  4.]
 [ 43.]
 [ 85.]
 [124.]
 [ 73.]
 [ 12.]
 [168.]
 [ 21.]
 [  9.]
 [  5.]
 [ 15.]
 [ 19.]
 [ 38.]
 [  2.]
 [ 70.]
 [154.]
 [ 12.]
 [242.]
 [  2.]
 [160.]
 [ 12.]
 [  2.]
 [  2.]
 [ 44.]
 [ 21.]
 [  4.]
 [  1.]
 [ 25.]
 [ 72.]
 [ 27.]
 [ 43.]
 [  3.]
 [  5.]
 [  2.]
 [ 46.]
 [ 28.]]


array([[0.        ],
       [0.30769231],
       [0.17857143],
       [0.13194444],
       [0.        ],
       [0.51162791],
       [0.24705882],
       [0.39516129],
       [0.32876712],
       [0.        ],
       [0.39880952],
       [0.38095238],
       [0.        ],
       [0.        ],
       [0.13333333],
       [0.47368421],
       [0.44736842],
       [0.        ],
       [0.3       ],
       [0.31818182],
       [0.25      ],
       [0.32644628],
       [0.        ],
       [0.30625   ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.38636364],
       [0.33333333],
       [0.        ],
       [0.        ],
       [0.2       ],
       [0.19444444],
       [0.66666667],
       [0.44186047],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.5       ],
       [0.32142857]])

In [40]:
print(confusion_matrix)
print(np.diag(confusion_matrix))
np.savetxt('NeuralNetworkres.txt', confusion_matrix)

[[0.99415888 0.         0.         ... 0.         0.         0.        ]
 [0.         0.89719626 0.         ... 0.         0.         0.        ]
 [0.         0.         0.95911215 ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.9953271  0.         0.        ]
 [0.         0.         0.         ... 0.         0.94976636 0.        ]
 [0.         0.         0.         ... 0.         0.         0.95443925]]
[0.99415888 0.89719626 0.95911215 0.80023364 0.9953271  0.94742991
 0.88434579 0.84813084 0.90771028 0.97897196 0.77453271 0.97196262
 0.98831776 0.99299065 0.98481308 0.98598131 0.95560748 0.99649533
 0.9135514  0.78621495 0.98014019 0.68341121 0.9953271  0.78037383
 0.98130841 0.99649533 0.9953271  0.94976636 0.97079439 0.99415888
 0.99883178 0.96495327 0.90303738 0.98364486 0.94275701 0.99182243
 0.99182243 0.9953271  0.94976636 0.95443925]
